# How to download flights csv file from transtats website

**In this notebook, we will**
1. Download a csv file for each of your chosen year(s) and month(s)
2. Prepare the data for further processing
3. Push the prepared data to a table in the database




In [49]:
# Import all necessary libraries
import pandas as pd
import numpy as np
import psycopg2 # needed to get database exception errors when uploading dataframe
import requests # package for getting data from the web
from zipfile import * # package for unzipping zip files

In [50]:
# Import the get_engine function from our sql_functions.
from sql_functions import get_engine #adjust this as necessary to match your sql_functions.py connection methods

# 1. Download csv file with flight data for your specific year/month

In the following, you are going to download a csv file containing flight data from [this website](https://transtats.bts.gov).    
You can specify, which data you want to download. 

Choose a month/year that you want to explore further.
With the following functions, you will download a csv file on public flight data from [this website](https://transtats.bts.gov) containing data of your chosen month/year.    
The file will be stored in a data folder.
Check out the url from which we download the data(https://transtats.bts.gov/PREZIP). Can we download data in this way from every source? What do you think?

In [5]:
# Specifies path for saving file
path ='data/' 
# Create the data folder
!mkdir {path}

mkdir: data/: File exists


In [6]:
# function to get specified csv file from the website https://transtats.bts.gov

def download_data(year, month):
    # Get the file from the website https://transtats.bts.gov
    zip_file = f'On_Time_Reporting_Carrier_On_Time_Performance_1987_present_{year}_{month}.zip'
    url = (f'https://transtats.bts.gov/PREZIP/{zip_file}')
    # Download the database
    r = requests.get(f'{url}', verify=False)
    # Save database to local file storage
    with open(path+zip_file, 'wb') as f:
        f.write(r.content)
        print(f'--> zip_file with name: {zip_file} downloaded succesfully.' )
    

In [7]:
# function to extract the csv files inside the zip files

def extract_zip(year, month):
    # Get the file from the website https://transtats.bts.gov
    zip_file = f'On_Time_Reporting_Carrier_On_Time_Performance_1987_present_{year}_{month}.zip'
    with ZipFile(path+zip_file, 'r') as zip_ref:
        zip_ref.extractall(path)
        csv_file =  zip_ref.namelist()[0]
        print(f'--> zip_file was succesfully extracted to: {csv_file}.' )

Don't worry - the following download of the data you chose may take some time ...

In [8]:
years_list = [2010] # list of years you want to look at (can of course also be a single year)
months_list = [2] # list of months you want to look at (can of course also be a single month)

# download flights data as zipfile(s)
# we use a nested loop to specify the years and months to define the range of the data we would like to have 
for year in years_list:
    for month in months_list:
        download_data(year, month)
        extract_zip(year, month)

/opt/homebrew/Caskroom/miniconda/base/envs/nf_sql/lib/python3.9/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'transtats.bts.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


--> zip_file with name: On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2010_2.zip downloaded succesfully.
--> zip_file was succesfully extracted to: On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2010_2.csv.


Now it is time to load the csv files into dataframes. You can create your own function equivalent to the functions above. But you need to decide whether...

- Do you need one dataframe for every month?
- Would you like to proceed with only one dataframe containing all the data you downloaded?
- One dataframe for every year?

There are certain things to consider before.
- changing column names
- dealing with missing data
- changing datatypes

In [9]:
# define the name of the csv file you want to read in
csv_file = 'On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2010_2.csv'

# Read in your data
df = pd.read_csv(path+csv_file, low_memory = False)
display(df.shape)
display(df.head())

(483270, 110)

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,Tail_Number,...,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Unnamed: 109
0,2010,1,2,2,2,2010-02-02,WN,19393,WN,N907WN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010,1,2,2,2,2010-02-02,WN,19393,WN,N270WN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010,1,2,2,2,2010-02-02,WN,19393,WN,N357SW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010,1,2,2,2,2010-02-02,WN,19393,WN,N918WN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010,1,2,2,2,2010-02-02,WN,19393,WN,N355SW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 2. Prepare the csv file for further processing

In the next step, we clean and prepare our dataset.

a) Since the dataset consists of a lot of columns, we we define which ones to keep.  
(Same as known from our SQL exercises on flights data.)

In [10]:
# Columns from downloaded file that are to be kept

columns_to_keep = [
                'FlightDate',
                'DepTime',
                'CRSDepTime',
                'DepDelay',
                'ArrTime',
                'CRSArrTime',
                'ArrDelay',
                'Reporting_Airline',
                'Tail_Number',
                'Flight_Number_Reporting_Airline',
                'Origin',
                'Dest',
                'AirTime',
                'ActualElapsedTime',
                'Distance',
                'Cancelled',
                'Diverted'
]

In [11]:
# The columns in the database have different naming as in the source csv files. Lets get the names from the database

schema = 'hh_analytics_24_1' # UPDATE 'TABLE_SCHEMA' based on schema used in class 
engine = get_engine() # assign engine to be able to query against the database

table_name_sql = f'''SELECT COLUMN_NAME 
                    FROM INFORMATION_SCHEMA.COLUMNS 
                    WHERE TABLE_NAME = 'flights'
                    AND TABLE_SCHEMA ='{schema}'
                    ORDER BY ordinal_position'''
c_names = engine.execute(table_name_sql).fetchall()
c_names

[('flight_date',),
 ('dep_time',),
 ('sched_dep_time',),
 ('dep_delay',),
 ('arr_time',),
 ('sched_arr_time',),
 ('arr_delay',),
 ('airline',),
 ('tail_number',),
 ('flight_number',),
 ('origin',),
 ('dest',),
 ('air_time',),
 ('actual_elapsed_time',),
 ('distance',),
 ('cancelled',),
 ('diverted',)]

In [12]:
# we can clean up the results into a clean list
new_column_names=[]
for name in c_names:
    new_column_names.append(name[0])
new_column_names        

['flight_date',
 'dep_time',
 'sched_dep_time',
 'dep_delay',
 'arr_time',
 'sched_arr_time',
 'arr_delay',
 'airline',
 'tail_number',
 'flight_number',
 'origin',
 'dest',
 'air_time',
 'actual_elapsed_time',
 'distance',
 'cancelled',
 'diverted']

In [13]:
# Just in case the above fails here are the results
'''new_column_names_alternate = ['flight_date', 'dep_time', 'sched_dep_time', 'dep_delay', 'arr_time', 'sched_arr_time', 
                'arr_delay', 'airline', 'tail_number', 'flight_number', 'origin', 'dest', 'air_time', 'actual_elapsed_time', 'distance', 'cancelled', 'diverted' ]'''

"new_column_names_alternate = ['flight_date', 'dep_time', 'sched_dep_time', 'dep_delay', 'arr_time', 'sched_arr_time', \n                'arr_delay', 'airline', 'tail_number', 'flight_number', 'origin', 'dest', 'air_time', 'actual_elapsed_time', 'distance', 'cancelled', 'diverted' ]"

b) With the next function, we make our csv file ready to be uploaded to SQL.  
We only keep to above specified columns and convert the datatypes.

In [14]:
def clean_airline_df(df):
    '''
    Transforms a df made from BTS csv file into a df that is ready to be uploaded to SQL
    Set rows=0 for no filtering
    '''

    # Build dataframe including only the columns you want to keep
    df_airline = df.loc[:,columns_to_keep]
     
    # Clean data types and NULLs
    df_airline['FlightDate']= pd.to_datetime(df_airline['FlightDate'], yearfirst=True)
    df_airline['CRSArrTime']= pd.to_numeric(df_airline['CRSArrTime'], downcast='integer', errors='coerce')
    df_airline['Cancelled']= pd.to_numeric(df_airline['Cancelled'], downcast='integer')
    df_airline['Diverted']= pd.to_numeric(df_airline['Diverted'], downcast='integer')
    df_airline['ActualElapsedTime']= pd.to_numeric(df_airline['ActualElapsedTime'], downcast='integer', errors='coerce')
    
    # Rename columns
    df_airline.columns = new_column_names
    
    return df_airline

In [15]:
# Call function and check resulting dataframe
df_clean = clean_airline_df(df)
df_clean.head()

,flight_date,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,airline,tail_number,flight_number,origin,dest,air_time,actual_elapsed_time,distance,cancelled,diverted
0,2010-02-02,830.0,830,0.0,842.0,900,-18.0,WN,N907WN,334,CLE,MDW,60.0,72.0,307.0,0,0
1,2010-02-02,1129.0,1135,-6.0,1134.0,1200,-26.0,WN,N270WN,356,CLE,MDW,55.0,65.0,307.0,0,0
2,2010-02-02,1424.0,1420,4.0,1443.0,1440,3.0,WN,N357SW,967,CLE,MDW,54.0,79.0,307.0,0,0
3,2010-02-02,625.0,635,-10.0,639.0,705,-26.0,WN,N918WN,1945,CLE,MDW,61.0,74.0,307.0,0,0
4,2010-02-02,1930.0,1820,70.0,1933.0,1845,48.0,WN,N355SW,2624,CLE,MDW,51.0,63.0,307.0,0,0


If you decide to only look at specific airports, it is a good decision to filter for them in advance.  
This function does the filtering. 

In [16]:
# Specify the airports you are interested in and put them as a list in the function.
def select_airport(df, airports):
    ''' Helper function for filtering the airline dataframe for a subset of airports'''
    df_out = df.loc[(df.origin.isin(airports)) | (df.dest.isin(airports))]
    return df_out


In [28]:
# Execute function, filtering for New York area airports
airports=['DCA', 'IAD', 'BWI', 'PHL', 'EWR', 'JFK', 'LGA']
if len(airports) > 0:
    df_selected_airports = select_airport(df_clean, airports)
else:
    df_selected_airports = df_clean
    
df_selected_airports.head()

,flight_date,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,airline,tail_number,flight_number,origin,dest,air_time,actual_elapsed_time,distance,cancelled,diverted
8,2010-02-02,1825.0,1830,-5.0,1926.0,1945,-19.0,WN,N257WN,484,CMH,BWI,50.0,61.0,336.0,0,0
9,2010-02-02,1047.0,1025,22.0,1153.0,1135,18.0,WN,N308SA,801,CMH,BWI,52.0,66.0,336.0,0,0
10,2010-02-02,1331.0,1335,-4.0,1434.0,1445,-11.0,WN,N751SW,1480,CMH,BWI,52.0,63.0,336.0,0,0
11,2010-02-02,638.0,630,8.0,749.0,740,9.0,WN,N942WN,3137,CMH,BWI,50.0,71.0,336.0,0,0
176,2010-02-02,949.0,955,-6.0,1453.0,1515,-22.0,WN,N799SW,146,DEN,BWI,173.0,184.0,1491.0,0,0


# 3. Push the prepared data to a table in the database

In [43]:
# Specify which table within your database you want to push your data to. Give your table an unambiguous name.
# Example: flights_sp for Sina's flights table, flights_groupname or similar
table_name = 'flights_api'
# If the specified table doesn't exist yet, it will be created
# With 'replace', your data will be replaced if the table already exists.
# This may take some time ...

# Write records stored in a dataframe to SQL database
if engine!=None:
    try:
        df_selected_airports.to_sql(name=table_name, # Name of SQL table
                        con=engine, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        schema=schema, # Use schmea that was defined earlier
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

In [54]:
from sql_functions import get_dataframe

In [57]:
schema = 'hh_analytics_24_1'

sql_query = f'SELECT * FROM {schema}.flights_group_x'
flights = get_dataframe(sql_query)

flights.head(20)

,flight_date,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,airline,tail_number,flight_number,...,air_time,actual_elapsed_time,distance,cancelled,diverted,lat,lon,city,tz_origin,tz_dest
0,2010-02-02,727.0,730,-3.0,835.0,855,-20.0,WN,N798SW,61,...,43.0,68.0,185.0,0,0,40.777199,-73.872597,New York,-5.0,-5.0
1,2010-02-02,1536.0,1540,-4.0,1646.0,1710,-24.0,WN,N210WN,65,...,40.0,70.0,185.0,0,0,40.777199,-73.872597,New York,-5.0,-5.0
2,2010-02-02,1239.0,1130,69.0,1338.0,1255,43.0,WN,N207WN,2064,...,42.0,59.0,185.0,0,0,40.777199,-73.872597,New York,-5.0,-5.0
3,2010-02-02,1355.0,1355,0.0,1514.0,1550,-36.0,WN,N456WN,232,...,111.0,139.0,725.0,0,0,40.777199,-73.872597,New York,-5.0,-6.0
4,2010-02-02,1026.0,1030,-4.0,1145.0,1225,-40.0,WN,N227WN,1510,...,116.0,139.0,725.0,0,0,40.777199,-73.872597,New York,-5.0,-6.0
5,2010-02-02,2007.0,1855,72.0,2124.0,2055,29.0,WN,N747SA,1903,...,113.0,137.0,725.0,0,0,40.777199,-73.872597,New York,-5.0,-6.0
6,2010-02-02,609.0,610,-1.0,734.0,800,-26.0,WN,N773SA,2190,...,122.0,145.0,725.0,0,0,40.777199,-73.872597,New York,-5.0,-6.0
7,2010-02-02,1742.0,1730,12.0,1900.0,1935,-35.0,WN,N780SW,3594,...,114.0,138.0,725.0,0,0,40.777199,-73.872597,New York,-5.0,-6.0
8,2010-02-02,1019.0,1015,4.0,1306.0,1325,-19.0,WN,N456WN,543,...,92.0,107.0,725.0,0,0,41.785999,-87.752403,Chicago,-6.0,-5.0
9,2010-02-02,747.0,750,-3.0,1107.0,1100,7.0,WN,N207WN,2064,...,93.0,140.0,725.0,0,0,41.785999,-87.752403,Chicago,-6.0,-5.0


In [36]:
# Just to be sure: Check if the number of rows match
table_name_sql = f'SELECT count(*) 
                    FROM {schema}.{table_name}
                    '
engine.execute(table_name_sql).fetchall()[0][0] == df_selected_airports.shape[0]

SyntaxError: EOL while scanning string literal (674723198.py, line 2)